In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from ResNet import ResNet18

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'CPU')
num_epoches = 80
batch_size = 100
learning_rate = 0.001

In [5]:
transform = transforms.Compose([        
        transforms.Pad(4),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32),
        transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True, 
                                             transform=transform,
                                             download=False)
test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False, 
                                            transform=transforms.ToTensor())
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


In [8]:
model = ResNet18().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [9]:
total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epoches):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 == 0:
            print("Epoch: [{}/{}], Step: [{}/{}], Loss: {:.4f}".format(
                epoch+1, num_epoches, i+1, total_step, loss.item()))
    
    if (epoch+1)%20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

Epoch: [1/80], Step: [100/500], Loss: 1.4997
Epoch: [1/80], Step: [200/500], Loss: 1.4918
Epoch: [1/80], Step: [300/500], Loss: 1.4879
Epoch: [1/80], Step: [400/500], Loss: 1.2800
Epoch: [1/80], Step: [500/500], Loss: 0.9247
Epoch: [2/80], Step: [100/500], Loss: 1.0509
Epoch: [2/80], Step: [200/500], Loss: 0.9940
Epoch: [2/80], Step: [300/500], Loss: 0.9750
Epoch: [2/80], Step: [400/500], Loss: 0.9140
Epoch: [2/80], Step: [500/500], Loss: 1.1025
Epoch: [3/80], Step: [100/500], Loss: 0.8133
Epoch: [3/80], Step: [200/500], Loss: 0.6732
Epoch: [3/80], Step: [300/500], Loss: 0.7865
Epoch: [3/80], Step: [400/500], Loss: 0.7446
Epoch: [3/80], Step: [500/500], Loss: 0.6880
Epoch: [4/80], Step: [100/500], Loss: 0.8727
Epoch: [4/80], Step: [200/500], Loss: 0.6936
Epoch: [4/80], Step: [300/500], Loss: 0.7664
Epoch: [4/80], Step: [400/500], Loss: 0.6504
Epoch: [4/80], Step: [500/500], Loss: 0.4993
Epoch: [5/80], Step: [100/500], Loss: 0.5321
Epoch: [5/80], Step: [200/500], Loss: 0.4501
Epoch: [5/

KeyboardInterrupt: 

In [10]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Accuracy of the model on the test images: {}%'.format(100*correct/total))

Accuracy of the model on the test images: 75.21%
